In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import json
import os
import sys

import helper

from selenium.common.exceptions import TimeoutException

import numpy as np

 # SETTING

In [2]:
NO_WINDOW = False
%run common_part_setting.ipynb

In [3]:
from tkinter import Y

day = DAY + 4

if MONTH in [1, 3, 5, 7, 8, 10, 12]:
    if day > 31:
        day = day % 31
elif MONTH in [4, 6, 9, 11]:
    if day > 30:
        day = day % 30
elif MONTH is "2":
    if YEAR % 4 != 0:
        if day > 28:
            day = day % 28
    else:
        if day > 29:
            day = day % 29

    
if day / 10 < 1:
    Day_string = "0{}".format(day)
else:
    Day_string = "{}".format(day)

# print(Day_string)

In [4]:
def create_new_driver():
    #setup option for chrome profile
    # chrome_options = Options()
    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    options = Options()
    
    if NO_WINDOW:
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')

    #start web driver
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.implicitly_wait(IMPLICIT_WAIT_TIME)

    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    return driver, wait

In [5]:
def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)

In [6]:
def sync_get_element_by_xpath(driver, xpath):
    wait = WebDriverWait(driver, 15)
    try:
        wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element_by_xpath(xpath)


In [7]:
def sign_in(driver, wait):
    
    
    username = driver.find_element_by_id('cphContents_txtUsername')
    password = driver.find_element_by_id('cphContents_txtPassword')

    username.send_keys("DN23035242")
    password.send_keys("NTW3BSFH")

    wait.until(ec.element_to_be_clickable((By.CLASS_NAME, "submit-button")))
#     wait.until(ec.presence_of_element_located((By.XPATH, "//div[@class='submit-button']/input")))

    sign_in_btn = driver.find_element_by_class_name("submit-button") 
    sign_in_btn.click()      
        

def log_out(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, "//a[contains(text(), 'Book for Class')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()

def click_book_this_class_now(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'BOOK THIS CLASS NOW')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
        return True
    else:
        return False

In [8]:
def scheduled_sign_in(driver, wait):
#     print("scheduled_sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)
    
def reserve_class(driver, wait, course_to_be_booked):
    
    print("reserve_class: ({})\n".format(course_to_be_booked['time']))
    
    book_url = course_to_be_booked['href']
    class_name = course_to_be_booked['name']
    
    reserve_class_log_dict = {}
    
    reserve_class_log_dict['name'] = class_name
    
    now1 = datetime.datetime.now()
    driver.get(book_url)
    now2 = datetime.datetime.now()

    now3 = datetime.datetime.now()
    now4 = datetime.datetime.now()
    if click_book_this_class_now(driver, wait):
        now4 = datetime.datetime.now()
    else:
        now4 = datetime.datetime.now()

    reserve_class_log_dict['datetime1'] = now1
    reserve_class_log_dict['datetime2'] = now2
    reserve_class_log_dict['datetime3'] = now3
    reserve_class_log_dict['datetime4'] = now4
    
    reserve_class_log_dicts.append(reserve_class_log_dict)
    
def show_all_booking_result(reserve_class_log_dicts):
    
    print("Scheduling Logs:")
    
    for reserve_class_log_dict in reserve_class_log_dicts:

        print("Class: ", reserve_class_log_dict['name'])
        print("-----------------------------------------\n")
        print("\t** Begin: Go to the class page at: {}\n".format(reserve_class_log_dict['datetime1']))
        print("\t** End: Go to the class page at: {}\n".format(reserve_class_log_dict['datetime2']))

        print("\t**Begin: Click 'BOOK THIS CLASS NOW' button at: {}\n".format(reserve_class_log_dict['datetime3']))
        print("\t**End: Click 'BOOK THIS CLASS NOW' button at: {}\n".format(reserve_class_log_dict['datetime4']))
        print("-----------------------------------------\n")

In [9]:
#launch url
# base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"
base_url = "https://www2.trueclassbooking.com.tw/member/search-class.aspx"

In [10]:
%run YEDN.ipynb
with open(schedule_filename, "r") as read_file:
    course_schedule = json.load(read_file)

[WDM] - Current google-chrome version is 114.0.5735
[WDM] - Get LATEST driver version for 114.0.5735
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/114.0.5735.90/chromedriver] found in cache


 
敦南 4 教室
敦南 6 教室
敦南 7 教室
敦南 9 教室
Time elapsed: 20.59


FileNotFoundError: [Errno 2] No such file or directory: '10 July - 16 July 2023.json'

### Place Test Code Block for Debugging here. The block is "stored" at the end of the notebook. Then commnet out the Production Code Block 

In [ ]:
# ....

### Production Code Block

In [ ]:
book_datetime = datetime.datetime(YEAR, MONTH, DAY, 22, 0, 0)

%run common_part_class.ipynb

helper.list_courses_to_be_booked(courses_to_be_booked)

# Set Sign In time 10 minutes before the open for booking time
signin_datetime = book_datetime - datetime.timedelta(seconds=TIME_INTERVAL_SIGNIN_BOOKING)

In [ ]:
book_datetime = book_datetime + datetime.timedelta(seconds=DELAY_TIME_BOOKING)

In [ ]:
reserve_class_log_dicts = []

In [ ]:
drivers = []
waits = []
scheduler = BackgroundScheduler()
start = time.time()
for idx, course_to_be_booked in enumerate(courses_to_be_booked):

    driver, wait = create_new_driver()
    driver.set_window_position(WINDOW_WIDTH * (idx % WINDOWS_GRID_X), WINDOW_HEIGHT * int(idx / WINDOWS_GRID_X));
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)
    signin_datetime_var = signin_datetime + datetime.timedelta(seconds=(idx+1)*2.5)
    print("idx: ", idx, "scheduled signin datetime: ", signin_datetime_var)
    print("idx: ", idx, "scheduled book datetime: ", book_datetime)
    
    scheduler.add_job(scheduled_sign_in, 'date', run_date=signin_datetime_var, args=[driver, wait])
    scheduler.add_job(reserve_class, 'date', run_date= book_datetime, args=[driver, wait, course_to_be_booked])
    
    if idx == len(courses_to_be_booked)-1:
        scheduler.add_job(show_all_booking_result, 'date', run_date= book_datetime + datetime.timedelta(seconds=6), args=[reserve_class_log_dicts])

# print("Time Elapsed: {:.2f}".format(time.time() - start)) 
scheduler.start()

In [ ]:
###### print("https://www.trueclassbooking.com.tw/member/login.aspx\n")
for idx, course_to_be_booked in enumerate(courses_to_be_booked):
    print("Class: {} \n\turl: {}\n".format(course_to_be_booked['name'], course_to_be_booked['href']))

### Test Code Block for Debugging

In [ ]:
#### NUM_CLASSES = 1
# DELAY_BETWEEN_CLASSSES = 3
# BASE_DELAY = 5
# TEST_SIGNIN_SECONDS_DELAY_FROM_NOW = BASE_DELAY
# TEST_BOOK_CLASSES_SECONDS_DELAY_FROM_NOW =  (NUM_CLASSES-1) * DELAY_BETWEEN_CLASSSES + BASE_DELAY*2

# signin_datetime = datetime.datetime.now() + datetime.timedelta(seconds=TEST_SIGNIN_SECONDS_DELAY_FROM_NOW)
# t = datetime.datetime.now()
# future_now = datetime.datetime(t.year, t.month, t.day, t.hour, t.minute + 1, 0)
# book_datetime = future_now + datetime.timedelta(seconds=TEST_BOOK_CLASSES_SECONDS_DELAY_FROM_NOW)

# print("Now: {}".format(datetime.datetime.now()))
# print("book_datetime: {}".format(book_datetime))
# courses_to_be_booked = []

# c = helper.search_courses_by_date('27', course_schedule)
# c = helper.search_courses_by_name('Mysore', c)
# # c = helper.search_courses_by_time('AM', c)


# courses_to_be_booked.extend(c)

# helper.list_courses_to_be_booked(courses_to_be_booked)